In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
)
def predict_category(query):
    prompt = f"""
    다음은 사용자의 질문입니다. 질문을 아래의 카테고리 중 하나로 분류해주세요:
    - 경마방법: 경마 규칙, 배팅 방법, 경마 용어 등과 관련된 질문.
    - 경기스케쥴: **날짜, 경마 일정, 시간표, 장소(예: 서울 경마공원 일정, 부산 경마공원 일정 등),** 2024년 경기와 관련된 질문. 예) "12월 21일 서울 경주 일정은?", "2024년 부산 경마 일정은?"
    - 우승마기록: 우승 기록, 인기 있는 말, 말의 성적 등과 관련된 질문.
    - 2024시행계획: 2024년 관련 계획, 정책, 새로운 규정 등과 관련된 질문.
    - 기타: 위의 카테고리에 해당하지 않는 질문.
    
    위 카테고리 중 가장 적합한 하나를 선택해서 답변해주세요. 위 카테고리에 포함되지 않으면 "기타"입니다.
    서울 경주 일정을 물어보면 경기스케줄로 분류하세요
"""
    response = model.invoke(prompt)
    category = response.content.strip()

    # 유효한 카테고리 확인
    valid_categories = ['경마방법', '경기스케쥴', '우승마기록', '2024시행계획']
    return category if category in valid_categories else "기타"

category_to_abbreviation = {
    '경마방법': 'guide',
    '경기스케쥴': 'schedule',
    '우승마기록': 'winners',
    '2024시행계획': 'plan2024',
    '기타': 'all', 
}

vectorstore_folder = 'C:/Workspace/DA36_mini4/min/vectorstores'
def ask_question_with_rag(query):
    predicted_category = predict_category(query)
    abbreviation = category_to_abbreviation.get(predicted_category, 'all')
    vectorstore_path = os.path.join(vectorstore_folder, f"vs_{abbreviation}")

    vector_store = FAISS.load_local(
        vectorstore_path, 
        OpenAIEmbeddings(model="text-embedding-ada-002"), 
        allow_dangerous_deserialization=True
    )
    retriever = vector_store.as_retriever()

    rqa = RetrievalQA.from_chain_type(
        llm=model,
        retriever=retriever,
        chain_type="stuff"
    )
    response = rqa({"query": query})
    return response['result']

### 카테고리 분류 질문

In [4]:
# 카테고리 질문
query="2024년 12월 21일 서울 1경주 경주력을 분석해서 배팅할만한 말 세 마리만 말해줘"
response = predict_category(query)
print(response)

경기스케쥴


### 테스트 QnA

In [3]:
query="12월21일 오후 4시 이후에 경기를 보러 갈 건데, 어떤 말에 배팅할까?"
print(ask_question_with_rag(query))

C:\Users\playdata2\AppData\Local\Temp\ipykernel_23824\394325305.py:47: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  OpenAIEmbeddings(model="text-embedding-ada-002"),
C:\Users\playdata2\AppData\Local\Temp\ipykernel_23824\394325305.py:57: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rqa({"query": query})


2024년 12월 21일 오후 4시 이후에 열리는 경주는 7경주 혼4 1200 핸디캡입니다. 이 경주에서 경주력이 좋은 말로는 ⑥ 문학홀리데이, ⑦ 원더풀웰스, ④ 관악산레오, ⑤ 제우시대 등이 있습니다. 이 말들 중에서 선택하여 배팅을 고려해보시면 좋을 것 같습니다.
